# Prep Notebook, Week 11

Here we'll redo some of our Altair plots, but now we'll be sure to save these in the appropriate place in our Jekyll template in order to have them show up on our project page.

We'll start with copying directly from the vega editor, just in case folks want to use that instead of Altair in Python.

## Setup -- Making a new project 

We'll first start by making a new project markdown file in order to add our visualizations.

We can do this by copying a file that is already there and them modifying for our new project:
1. Project name
1. the "tools" tag
1. the image (we might wait to do this until we can take a screenshot of one of our new plots)
1. the short description

Note: we want to make sure to keep all the `custom_js` tags in order to be able to include our interactive creations!

## 1. Copying directly from vega-editor

We can start with saving vega-lite code directly as json if we do our development in the vega-editor.

If we start from this [simple vega-example](https://vega.github.io/vega-lite/examples/stacked_bar_h.html) and [open it in the vega-editor](https://vega.github.io/editor/#/examples/vega-lite/stacked_bar_h) we can export from the vega-lite editor.

If we click the Export button in the upper tool bar, we see that we have a few options for how to export visualizations.

For this to work, you need to make sure that you have the full raw URL for the dataset from the [list of vega-datasets](https://github.com/vega/vega-datasets/tree/master/data). Double check that it is the raw-data link!

![click on the "Export" button in the vega editor](images/vegaeditor/vega_editor1.png)


We can save with JSON (config file un-checked) using the middle upper panel's interface:


![make sure "include config" is checked](images/vegaeditor/vega_editor2.png)


We can then save the downloaded .json file in the assets/jsons/ folder in our main Jekyll page directory.

Once we have that we can link using vegachart in your Jekyll project page:

```html
<vegachart schema-url="{{ site.baseurl }}/assets/json/direct_from_editor.json" style="width: 100%"></vegachart>
```


## 2. Save from Python -- data at URL

In [1]:
import pandas as pd
import numpy as np
import altair as alt

Let's use our Mobility dataset from last time and remake our dashboard plot:

In [2]:
mobility_url = 'https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/mobility.csv'

In [3]:
brush = alt.selection_interval(encodings=['x','y'])

chart1 = alt.Chart(mobility_url).mark_rect().encode(
    alt.X("Student_teacher_ratio:Q", bin=alt.Bin(maxbins=10)),
    alt.Y("State:O"),
    alt.Color("count()")
).properties(
   height=400
).add_selection(
        brush
)

chart2 = alt.Chart(mobility_url).mark_bar().encode(
    alt.X("Mobility:Q", bin=True,axis=alt.Axis(title='Mobility Score')),
    alt.Y('count()', axis=alt.Axis(title='Mobility Score Distribution'))
).transform_filter(
    brush
)

chart = (chart1.properties(width=300) | chart2.properties(width=300))

chart

/var/folders/t7/bwcvd_4177q4872gxghn7p9r0000gq/T/ipykernel_2959/1178754509.py:9: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.HConcatChart(...)

Now that we have this plot, we want to save it in a place we can access from our Jekyll template.  This place, like with copying from our vega-editor, is in `<YOUR GITHUB PAGES DIRECTORY>/assets/json`.

If working locally, we want to make sure we save this in this directory.

If working on PrairieLearn, we just need to save the json somewhere and then make sure we download and move it to the `assets/json` folder in our template:

In [4]:
# from PL:
# myJekyllDir = "./"
# --> then be sure to download these jsons to your "asset/json" folder with a right-click in PL!

# working locally:
myJekyllDir = '/Users/jnaiman/jnaiman.github.io/assets/json/'

Now that we have the path set, let's save our chart:

In [5]:
chart.save(myJekyllDir+"altair_mobility_dashboard.json")

### Saving within a "container"

The mobility dashboard is a "faceted" chart, since it is made up of two plots.  As of last I checked, you can't save faceted charts within a variable "container" that will update if the user changes the size of the window (i.e. if on mobile device, or just changing the size of a browser window).

However, for single-facet charts, you can save in a container that is variable.  For example:

In [6]:
scatter = alt.Chart(mobility_url).mark_point().encode(
    x='Mobility:Q', # "Q for quantiative"
    y=alt.Y('Population:Q', scale=alt.Scale(type='log')),
    color=alt.Color('Income:Q', scale=alt.Scale(scheme='sinebow'),bin=alt.Bin(maxbins=5))
)
scatter

alt.Chart(...)

In [7]:
scatter.properties(width='container').save(myJekyllDir+"population_scatter.json")

The above will now resize (within reason) if your browser window changes.

Support for multi-faceted charts is on the development pathway within vegalite, so hopefully that will be supported in the near future.